# Péndulos acoplados

Federico Cerisola (cerisola@df.uba.ar)

Departamento de Física, Facultad de Ciencias Exactas y Naturales, Universidad de Buenos Aires

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

from matplotlib import animation, rc
from IPython.display import HTML
import ipywidgets as widgets

rc('animation', html='jshtml')

In [3]:
def calculo_modos_normales(mA, mB, k, l, g=9.8):
    omega1 = np.sqrt(g/l)
    omega2 = np.sqrt(g/l + k*(mA + mB)/(mA*mB))
    v1 = np.array([1, 1])
    v2 = np.array([1, -mA/mB])
    return (omega1, v1), (omega2, v2)


def calculo_ci(psiA0, psiB0, velA0, velB0, modos):
    (omega1, v1), (omega2, v2) = modos
    A = np.array([[v1[0], 0, v2[0], 0],
                  [v1[1], 0, v2[1], 0],
                  [0, omega1*v1[0], 0, omega2*v2[0]],
                  [0, omega1*v1[1], 0, omega2*v2[1]]])
    d = np.array([psiA0, psiB0, velA0, velB0])
    Amplitudes = linalg.solve(A, d)
    C1 = np.sqrt(Amplitudes[0]**2 + Amplitudes[1]**2)
    C2 = np.sqrt(Amplitudes[2]**2 + Amplitudes[3]**2)
    phi1 = np.arctan2(-Amplitudes[1], Amplitudes[0])
    phi2 = np.arctan2(-Amplitudes[3], Amplitudes[2])
    return (C1, phi1), (C2, phi2)
    #return ((Amplitudes[0], Amplitudes[1]), (Amplitudes[2], Amplitudes[3]))


def funciones_posicion_masas(modos, CI):
    (omega1, v1), (omega2, v2) = modos
    (C1, phi1), (C2, phi2) = CI

    psiA = lambda t: v1[0]*C1*np.cos(omega1*t + phi1) + v2[0]*C2*np.cos(omega2*t + phi2)
    psiB = lambda t: v1[1]*C1*np.cos(omega1*t + phi1) + v2[1]*C2*np.cos(omega2*t + phi2)
    return psiA, psiB

In [4]:
def graficar_soluciones(tmax, mA, mB, k, l, psiA0, psiB0, velA0, velB0, g=9.8):
    modos = calculo_modos_normales(mA, mB, k, l, g=g)
    CI = calculo_ci(psiA0, psiB0, velA0, velB0, modos)
    psiA, psiB = funciones_posicion_masas(modos, CI)

    t = np.linspace(0, tmax, 4000)

    plt.figure(figsize=(9,5))
    plt.plot(t, psiA(t), label='$\psi_{A}$')
    plt.plot(t, psiB(t), label='$\psi_{B}$')
    plt.xlabel('$t$')
    plt.ylabel('Desplazamiento')
    plt.grid()
    plt.legend()
    plt.show()

In [5]:
# creamos widget iteractivo
widgets.interact(graficar_soluciones,
                tmax=widgets.FloatSlider(min=0, max=500, step=1, value=100),
                mA=widgets.FloatSlider(min=0, max=10, step=0.1, value=1),
                mB=widgets.FloatSlider(min=0, max=10, step=0.1, value=1),
                k=widgets.FloatSlider(min=0, max=10, step=0.1, value=0.5),
                l=widgets.FloatSlider(min=0, max=10, step=0.1, value=1),
                psiA0=widgets.FloatSlider(min=-1, max=1, step=0.1, value=1),
                psiB0=widgets.FloatSlider(min=-1, max=1, step=0.1, value=0),
                velA0=widgets.FloatSlider(min=-1, max=1, step=0.1, value=0),
                velB0=widgets.FloatSlider(min=-1, max=1, step=0.1, value=0))

interactive(children=(FloatSlider(value=100.0, description='tmax', max=500.0, step=1.0), FloatSlider(value=1.0…

<function __main__.graficar_soluciones(tmax, mA, mB, k, l, psiA0, psiB0, velA0, velB0, g=9.8)>

In [16]:
def animacion_soluciones(tmax, mA, mB, k, l, psiA0, psiB0, velA0, velB0, g=9.8):
    modos = calculo_modos_normales(mA, mB, k, l, g=g)
    CI = calculo_ci(psiA0, psiB0, velA0, velB0, modos)
    psiA, psiB = funciones_posicion_masas(modos, CI)

    t = np.linspace(0, tmax, 400)
    psiAt = psiA(t)
    psiBt = psiB(t)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))
    lineA, = ax1.plot([], [], label='$\psi_{A}$')
    lineB, = ax1.plot([], [], label='$\psi_{A}$')
    ax1.set_xlabel('$t$')
    ax1.set_ylabel('Despalazamiento')
    
    L = np.max([psiAt, psiBt])
    posEqA = L
    posEqB = 3*L
    ax2.set_xlim([0, 4*L])
    ax2.set_ylim([0, 1])
    lineTecho, = ax2.plot([0, 4*L], [0.9, 0.9], '-', color='black', linewidth=3.0)
    linePendA, = ax2.plot([posEqA, posEqA], [0.9, 0.1], '-', color='C0', linewidth=2.0)
    linePendB, = ax2.plot([posEqB, posEqB], [0.9, 0.1], '-', color='C1', linewidth=2.0)
    lineResor, = ax2.plot([posEqA, posEqB], [0.1, 0.1], '--', color='gray', linewidth=1.5)
    masaPendA, = ax2.plot([posEqA], [0.1], 'o', color='C0', markersize=10.0)
    masaPendB, = ax2.plot([posEqB], [0.1], 'o', color='C1', markersize=10.0)
    
    def init():
        lineA.set_data([], [])
        lineB.set_data([], [])
        linePendA.set_xdata([posEqA, posEqA])
        linePendB.set_xdata([posEqB, posEqB])
        lineResor.set_xdata([posEqA, posEqB])
        masaPendA.set_xdata([posEqA])
        masaPendB.set_xdata([posEqB])
        return lineA, lineB, linePendA, linePendB, masaPendA, masaPendB
    
    def animate(i):
        lineA.set_data(t[:i], psiAt[:i] + posEqA)
        lineB.set_data(t[:i], psiBt[:i] + posEqB)
        linePendA.set_xdata([posEqA, psiAt[i] + posEqA])
        linePendB.set_xdata([posEqB, psiBt[i] + posEqB])
        lineResor.set_xdata([psiAt[i] + posEqA, psiBt[i] + posEqB])
        masaPendA.set_xdata([psiAt[i] + posEqA])
        masaPendB.set_xdata([psiBt[i] + posEqB])
        return lineA, lineB, linePendA, linePendB, masaPendA, masaPendB 

    anim = animation.FuncAnimation(fig, animate, frames=len(t), interval=100, blit=True, init_func=init)
    #HTML(anim.to_jshtml())
    return anim

In [17]:
# creamos widget iteractivo
widgets.interact(animacion_soluciones,
                tmax=widgets.FloatSlider(min=0, max=500, step=1, value=50, continuous_update=False),
                mA=widgets.FloatSlider(min=0, max=10, step=0.1, value=1, continuous_update=False),
                mB=widgets.FloatSlider(min=0, max=10, step=0.1, value=1, continuous_update=False),
                k=widgets.FloatSlider(min=0, max=10, step=0.1, value=0.5, continuous_update=False),
                l=widgets.FloatSlider(min=0, max=10, step=0.1, value=1, continuous_update=False),
                psiA0=widgets.FloatSlider(min=-1, max=1, step=0.1, value=1, continuous_update=False),
                psiB0=widgets.FloatSlider(min=-1, max=1, step=0.1, value=0, continuous_update=False),
                velA0=widgets.FloatSlider(min=-1, max=1, step=0.1, value=0, continuous_update=False),
                velB0=widgets.FloatSlider(min=-1, max=1, step=0.1, value=0, continuous_update=False))

interactive(children=(FloatSlider(value=50.0, continuous_update=False, description='tmax', max=500.0, step=1.0…

<function __main__.animacion_soluciones(tmax, mA, mB, k, l, psiA0, psiB0, velA0, velB0, g=9.8)>